# Assignment3 </br>
### Orly Olbum </br>


## P. 30 Problem 1 </br>
### Tokenize the following sentence:  </br>
### After sleeping for 2h, he decided to sleep for another two.

In [45]:
sentence1 = "After sleeping for 2h, he decided to sleep for another two."

def tokenizeWithWhiteSpace(doc: str):
    '''Tokenize a document only using white space,
      Args
        doc -- is one string to be parsed
      Returns:
        a list with elements being the string split by white space
    '''
    return doc.split()

token1 = tokenizeWithWhiteSpace(sentence1)
print(token1)

['After', 'sleeping', 'for', '2h,', 'he', 'decided', 'to', 'sleep', 'for', 'another', 'two.']


## P. 30 Problem 2 </br>
### Assume that all article, pronouns, and prepositions are stop words. Perform a sensible stemming and case folding in the example of Exercise 1, and convert to a vector-space representation. Express your representation as a set of words with associated frequencies but no normalization.

In [46]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\orlyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\orlyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
# print words that are not stop words
tokens_without_sw = [word for word in token1 if not word in stopwords.words()]
print(tokens_without_sw)

['After', 'sleeping', '2h,', 'decided', 'sleep', 'another', 'two.']


In [48]:
# case folding: proper case
def lowerIt(tokenList):
    '''Lower case all tokens,
      input:
        tokenList: list of tokens 
      output:
        a list of lower case tokens
    '''
    return [i.lower() for i in tokenList]

lower = lowerIt(tokens_without_sw)
print(lower)

['after', 'sleeping', '2h,', 'decided', 'sleep', 'another', 'two.']


In [49]:
# stemming: consolidate words with the same root
# Lets stem
myPatts = [r"(?<=.{2})ies\Z", r"es\Z", r"(?<![ha|wa|$i|$a])(s{1})\Z", r"(?<=.{2})(ing)\Z", r"ly\Z", r"er\Z",r"(?<=.{2})(ed)\Z"]
# note 2: r"(?<![$ha|$wa|$i|$a])(s{1})\Z" uses negative look behind to avoid trimming ("has", "was", "is", "as") it would leave wawas
# other patterns make sure that words like ring and red do not get stemmed (the "".{2}"" part)
# (?<=) is a positive lookbehind. it says to look to right of the pattern in the group (), and 
# do not match that patter, just match the regex after it. Here, I am making sure there are at least two characters
# before we lop off an ing or ed.

def myStemmer(myList, myPatts):
    ''' Stem tokens in myList based on myPatts
    
    Args
      myList -- list of tokens to stem
      myPatts -- list of regex patterns to use for stemming
        each pattern, if matched, lead to a deletion
    Returns
      A list of stemmed tokens
      
    Note: the order of patterns matters, eg taking `ing` off changes the end of the
      token and thus the regex match
    '''    
    out = myList
    for pat in myPatts:
        out = [re.sub(pat, "", i) for i in out]
    return out

print(set(myStemmer(lower, myPatts)))
per_change = round(100-len(set(myStemmer(lower, myPatts)))/len(set(lower)) *100, ndigits=1)
print(f"\n {per_change} percent fewer terms".format( ))

{'2h,', 'two.', 'anoth', 'decid', 'sleep', 'aft'}

 14.3 percent fewer terms


In [50]:
# vector space representation
from collections import Counter

def myDocTFVectorizor(doc, returnList=False):
    '''Create dictionary (or list) of term frequencies from a list of tokens in a doc
    Args
      doc: a list of tokens for the document
      returnList: do you want a dict (default) or list (returnList=True)
    Returns
      Either a dict with key as term and count as the value
      or a list of tuples with the first val the term, and second the count
    '''
    # Counter is like a dict, but we can increment values to keys
    # that do not exist
    out = Counter()
    for token in doc:
        # this is the magic, increment even if token was not seen yet!
        out[token] += 1
    if not returnList:
        return out
    # some people like lists, so there is that option too
    elif returnList:
        return [(key,val) for key,val in out.items()]

#Keep order of terms
output1 = myDocTFVectorizor(lower)
output1

Counter({'after': 1,
         'sleeping': 1,
         '2h,': 1,
         'decided': 1,
         'sleep': 1,
         'another': 1,
         'two.': 1})

In [51]:
output2 = myDocTFVectorizor(lower, returnList=True)
myVocab, myCounts = zip(*output2)
print(list(zip(myVocab, myCounts)))

[('after', 1), ('sleeping', 1), ('2h,', 1), ('decided', 1), ('sleep', 1), ('another', 1), ('two.', 1)]


## P. 30 Problem 3 </br>
### Consider a collection in which the words "after," "decided," and "another," each occur in 16% of the documents. All other words occur in 4% of the documents. Create and idf-normalized representation of your answer in Exercise 2.

In [52]:
# put lists into a dictionary
original = {myVocab[i]: myCounts[i] for i in range(len(myVocab))}
print(original)

{'after': 1, 'sleeping': 1, '2h,': 1, 'decided': 1, 'sleep': 1, 'another': 1, 'two.': 1}


In [53]:
freq_list = ["after", "decided", "another"]

# for keys from the original that are in the freq_list, multiply their respective values by 0.16
# for all other keys that are not in the freq_list, multiply their values by 0.04
for key in original:
    if key in freq_list:
        original[key] *= 0.16
    else:
        original[key] *= 0.04

# normalized values
print(original)

{'after': 0.16, 'sleeping': 0.04, '2h,': 0.04, 'decided': 0.16, 'sleep': 0.04, 'another': 0.16, 'two.': 0.04}


## P. 30 Problem 5 </br>
### Compute the cosine similarity between the vector pair (1,2,3,4,0,1,0) and (4,3,2,1,1,0,0). Repeat the same computation with the Jaccard coefficient.

In [54]:
import numpy as np

vector1 = [1, 2, 3, 4, 0, 1, 0]
vector2 = [4, 3, 2, 1, 1, 0, 0]

# from lecture: cosine similarity
def myCosineSim(vec1, vec2):
    import math
    import numpy as np
    vec1n = np.array(vec1)
    vec2n = np.array(vec2)
    return (vec1n * vec2n).sum() / (math.sqrt(((vec1n)**2).sum())*math.sqrt(((vec2n)**2).sum()))

myCosineSim(vector1, vector2)

0.6451612903225807

In [55]:
# from lecture: Jaccard coefficient

def jaccard(b1, b2):
    '''Calculate Jaccard index for two binary vector
    expects np.ndarrays
    '''
    x = np.array(b1)
    y = np.array(b2)
    return (x*y).sum()/(x.sum(0) + y.sum(0) - (x*y).sum(0))

jaccard(vector1, vector2)

10.0

## P. 30 Problem 6 </br>
### Normalize each of the vectors in Exercise 5 to unit norm. Compute the Euclidean distance between the pair of normalized vectors. What is the relationship between this Euclidean distance and the cosine similarity computed in Exercise 5?

In [56]:
# normalize the vectors to unit norm
v1 = np.array(vector1)
v2 = np.array(vector2)

vector1_norm = v1 / np.sqrt(np.sum(v1 ** 2))
vector2_norm = v2 / np.sqrt(np.sum(v2 ** 2))

In [57]:
# from lecture: Euclidean distance
def myEuclideanDist(vec1, vec2):
    import math
    import numpy as np
    vec1n = np.array(vec1)
    vec2n = np.array(vec2)
    return math.sqrt(((vec1n-vec2n)**2).sum())

myEuclideanDist(vector1_norm, vector2_norm)

0.8424235391742319

### While the Euclidean distance is the distance between two vectors calculated using their norms (difference), the cosine similarity is the distance between two vectors with respect to their magnitudes. Euclidean distance serves better for text classification whereas cosine similarity functions better for retrieving texts similar to the document of interest.